In [35]:
from os.path import abspath
import numpy as np

In [36]:
def load_data(path):
    input_file = open(path)
    output_data = []
    for line in input_file:
        one_row = line.split(',')
        output_data.append([float(x) for x in one_row])

    print('Input data info.')
    print('\tFeatures amount: %d, set size %d' % (len(output_data[0]), len(output_data)))
    return np.array(output_data)

In [37]:
def show_prediction(output_vector):
    index_of_max = np.argmax(np.roll(output_vector[1:], 1))
    return index_of_max

In [38]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [39]:
# sum of all pairs theta[i]*input[i] in one neuron
# z(i) = sum(theta * x) = theta' * x
def unit_summarization(theta, input_vec):
    return (np.array(theta) * np.array(input_vec)).sum()

In [40]:
# computing the activation function of neuron
# a(i) = g(z(i))
def unit_activation(theta, input_vec):
    sum_result = unit_summarization(theta, input_vec)
    return sigmoid(sum_result)

In [41]:
# computing the activation vector of whole network layer
# A = g(z(i)) for all units on layer
# [1] - additional neuron (bias)
def forward_propagation_step(theta_matrix, input_vec):
    activations_vector = [unit_activation(theta_matrix[i], input_vec) for i in range(len(theta_matrix))] 
    return np.concatenate(([1.0], activations_vector))

In [42]:
def forward_propagation(theta_one, theta_two, input_vec):
    # computing input level activation vector
    activation_vector_input_layer = np.concatenate(([1.0], input_vec))
    # computing second level activation vector
    activation_vector_hidden_layer = forward_propagation_step(theta_one, activation_vector_input_layer)
    # computing output level activation vector
    activation_vector_output_layer = forward_propagation_step(theta_two, activation_vector_hidden_layer)
    return [activation_vector_hidden_layer, activation_vector_output_layer]

In [43]:
def compute_regularize_coefficient(lm, m, theta):
    coefficient = lm / (2.0 * m)
    a = np.array(theta[0]) * np.array(theta[0])
    b = np.array(theta[1]) * np.array(theta[1])
    return (a.sum() + b.sum()) * coefficient

In [44]:
def get_y(i, k):
    offset = 500
    return 1.0 if k * offset <= i < k * offset + offset else 0.0


def get_output_vector(num):
    out_vec = [0.0] * 10
    out_vec[int(num - 1)] = 1.0
    return [1.0] + out_vec

In [45]:
def compute_cost_function(x_data, theta, lm):
    temp_sum = 0.0
    coefficient = -1 / len(x_data)
    fake_zero = 10 ** (-9)
    
    regularize_coefficient = compute_regularize_coefficient(lm, len(x_data), theta)
    for k in range(10):
        for i in range(len(x_data)):
            hyp_vector = forward_propagation(theta[0], theta[1], x_data[i])[1]
            hyp = np.roll(hyp_vector[1:], 1)[k]
            temp_sum += get_y(i, k) * np.log(hyp if hyp != 0.0 else fake_zero) \
                + (1.0 - get_y(i, k)) * np.log(1 - hyp if 1 - hyp != 0.0 else fake_zero)
 
    return temp_sum * coefficient + regularize_coefficient

In [48]:
# for all layers except last one
def compute_layer_error(layer_delta, theta):
    return theta.transpose() @ layer_delta

In [49]:
def compute_layer_delta(layer_error, layer_activation):
    layer_error = np.array(layer_error)
    return (layer_error * layer_activation * (1.0 - layer_activation))[1:]

In [50]:
def unit_theta_correction(prev_layer_activation, delta, rate):
    return prev_layer_activation * delta * rate

In [51]:
def layer_theta_correction(prev_layer_activation, layer_delta, lr, add_neuron=False):
    prev_layer_activation = np.concatenate(([[1.0], prev_layer_activation])) if add_neuron else prev_layer_activation
    return [unit_theta_correction(prev_layer_activation, layer_delta[i], lr) for i in range(len(layer_delta))]

In [52]:
def back_propagation(x_data, theta, learning_rate):
    theta_one = np.array(theta[0])
    theta_two = np.array(theta[1])

    for i in range(len(x_data)):
        # running forward propagation for getting hypothesis  
        layer_activations = forward_propagation(theta_one, theta_two, x_data[i])

        # getting outputs of hidden and output layer
        output_layer_activation = layer_activations[1]
        hidden_layer_activation = layer_activations[0]

        ############### output layer section ############
        # computing output layer error, it's different for all other layers 
        output_layer_error = output_layer_activation - get_output_vector(i / 500)
        # computing deltas, it will be used for computing new theta (new weights)
        # delta = error * sigmoid(x)dx
        output_layer_delta = compute_layer_delta(output_layer_error, output_layer_activation)
        # correcting theta
        theta_two -= np.array(layer_theta_correction(hidden_layer_activation, output_layer_delta, learning_rate))

        # ############### hidden layer section ############
        hidden_layer_error = compute_layer_error(output_layer_delta, theta_two)
        hidden_layer_delta = compute_layer_delta(hidden_layer_error, hidden_layer_activation)
        theta_one -= np.array(layer_theta_correction(x_data[i], hidden_layer_delta, learning_rate, True))
        
    return [theta_one, theta_two]


In [53]:
loaded_data_x = load_data(abspath('demo/init/coursera/neuralnetwork/data/data_x.csv'))
loaded_data_y = load_data(abspath('demo/init/coursera/neuralnetwork/data/data_y.csv'))
loaded_theta_one = load_data(abspath('demo/init/coursera/neuralnetwork/data/theta1.csv'))
loaded_theta_two = load_data(abspath('demo/init/coursera/neuralnetwork/data/theta2.csv'))

Input data info.
	Features amount: 400, set size 5000
Input data info.
	Features amount: 1, set size 5000
Input data info.
	Features amount: 401, set size 25
Input data info.
	Features amount: 26, set size 10


In [54]:
def check_network(expected_theta, actual_theta, x_data):
    expected = forward_propagation(expected_theta[0], expected_theta[1], x_data)[1]
    actual = forward_propagation(actual_theta[0], actual_theta[1], x_data)[1]
    print('Expected: %d  Actual: %d' % (show_prediction(expected), show_prediction(actual)))
    print('Output: ', list(actual[1:]))

In [55]:
def run():
    features_amount = 400
    hidden_layer_neuron_amount = 25
    hidden_layer_inputs_per_neuron = features_amount + 1
    output_layer_neuron_amount = 10
    output_layer_inputs_per_neuron = hidden_layer_neuron_amount + 1
    
    learning_rate = 0.01
    epochs = 200
    
    computed_thetas = [[], []]
    # initializing thetas (weights of each connection between neurons)
    # weights between input and hidden layer
    computed_thetas[0] = np.random.rand(hidden_layer_neuron_amount, hidden_layer_inputs_per_neuron) / 100
    # weights between hidden and output layer
    computed_thetas[1] = np.random.rand(output_layer_neuron_amount, output_layer_inputs_per_neuron) / 100

    for i in range(epochs):
        if i % 10 == 0:
            print('Epochs left', epochs - i)
        computed_thetas = back_propagation(loaded_data_x, computed_thetas, learning_rate) 
        
    return computed_thetas

cool_thetas = run()

[check_network([loaded_theta_one, loaded_theta_two], cool_thetas, loaded_data_x[j * 500]) for j in range(10)]

Epochs left 200


Epochs left 190


Epochs left 180


Epochs left 170


Epochs left 160


...

Epochs left 30


Epochs left 20


Epochs left 10


Expected: 0  Actual: 1
Output:  [0.99121418609407208, 0.0017610407981747925, 0.0023183744138181484, 8.0462640363083303e-07, 0.013654034012992861, 0.00019043141178000459, 0.0016149095317928922, 0.00025323160865909895, 1.5352015814622589e-05, 0.00063232322633801936]
Expected: 1  Actual: 1
Output:  [0.89831576875075703, 0.069769395290902883, 0.0022844766812163565, 0.00014518608472565954, 0.0010611754391410177, 0.00041597637326421731, 0.0025197711065866918, 0.02447391223359727, 2.5057160909754739e-05, 0.001750608777421648]
Expected: 2  Actual: 2
Output:  [0.00070701932153999735, 0.81152429628204747, 0.0085174343899941853, 8.1520039005223061e-05, 0.00029434527348088394, 8.2184828863994812e-06, 0.00084219220231076316, 0.088068595651955933, 0.027697387733959966, 0.0013551287569565536]
Expected: 3  Actual: 3
Output:  [0.0010984054122966356, 0.0020312749733667366, 0.9850275244216381, 1.8237449320348592e-05, 0.0054097272552923893, 5.9826322871933931e-07, 0.0011441105836785574, 0.0069309986116280

[None, None, None, None, None, None, None, None, None, None]